# Integrated Farming Decisions: Comprehensive Case Study

## Real-World Scenario

You're planning the upcoming growing season for a 200-acre farm. Multiple uncertain factors affect profitability:
- Weather patterns
- Disease risks
- Market prices
- Input costs

**Key Decisions:**
1. Which crop variety to plant?
2. Whether to invest in irrigation?
3. How much crop insurance to buy?
4. When to plant and harvest?

This notebook integrates ALL probability concepts from this module into one comprehensive analysis.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from scipy import stats
import sys
sys.path.append('../2_from_scratch')
import probability_functions as pf

plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
np.random.seed(42)

print("✓ Ready for comprehensive farm planning!")

## Decision 1: Crop Variety Selection

**Three varieties available:**
- Variety A: High yield potential, disease-susceptible
- Variety B: Moderate yield, disease-resistant  
- Variety C: Lower yield, drought-tolerant

In [ ]:
# Crop variety characteristics
varieties = {
    'A': {'base_yield': 120, 'disease_susceptible': 0.25, 'drought_susceptible': 0.40},
    'B': {'base_yield': 100, 'disease_susceptible': 0.10, 'drought_susceptible': 0.30},
    'C': {'base_yield': 85, 'disease_susceptible': 0.15, 'drought_susceptible': 0.10}
}

# Environmental probabilities
P_disease_outbreak = 0.12
P_drought = 0.20

# Yield reductions if problems occur
disease_yield_factor = 0.60  # 60% yield if diseased
drought_yield_factor = 0.70  # 70% yield if drought

print("Variety Selection Analysis")
print("="*80)
print(f"\nEnvironmental Risks:")
print(f"  P(Disease Outbreak) = {P_disease_outbreak:.0%}")
print(f"  P(Drought) = {P_drought:.0%}")

# Calculate expected yield for each variety
results = []

for name, char in varieties.items():
    base_yield = char['base_yield']
    
    # P(damage) = P(disease AND susceptible) OR P(drought AND susceptible)
    p_disease_damage = P_disease_outbreak * char['disease_susceptible']
    p_drought_damage = P_drought * char['drought_susceptible']
    
    # Assuming disease and drought are independent
    p_both = p_disease_damage * p_drought_damage
    p_disease_only = p_disease_damage * (1 - p_drought_damage)
    p_drought_only = p_drought_damage * (1 - p_disease_damage)
    p_neither = (1 - p_disease_damage) * (1 - p_drought_damage)
    
    # Calculate expected yield
    exp_yield = (
        p_neither * base_yield +
        p_disease_only * (base_yield * disease_yield_factor) +
        p_drought_only * (base_yield * drought_yield_factor) +
        p_both * (base_yield * disease_yield_factor * drought_yield_factor)
    )
    
    results.append({
        'Variety': name,
        'Base Yield': base_yield,
        'P(Disease Damage)': p_disease_damage,
        'P(Drought Damage)': p_drought_damage,
        'Expected Yield': exp_yield
    })

df_varieties = pd.DataFrame(results)
print("\n" + df_varieties.to_string(index=False))

best_variety = df_varieties.loc[df_varieties['Expected Yield'].idxmax(), 'Variety']
best_yield = df_varieties['Expected Yield'].max()

print(f"\n✓ RECOMMENDATION: Variety {best_variety} (Expected: {best_yield:.1f} bu/ac)")

# Visualize
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

varieties_list = df_varieties['Variety'].values
base_yields = df_varieties['Base Yield'].values
exp_yields = df_varieties['Expected Yield'].values

x = np.arange(len(varieties_list))
width = 0.35

ax1.bar(x - width/2, base_yields, width, label='Base Yield', color='lightgreen', edgecolor='black')
ax1.bar(x + width/2, exp_yields, width, label='Expected Yield', color='darkgreen', edgecolor='black')
ax1.set_xticks(x)
ax1.set_xticklabels(varieties_list)
ax1.set_ylabel('Yield (bu/ac)')
ax1.set_title('Base vs Expected Yield')
ax1.legend()

risk_scores = df_varieties['P(Disease Damage)'] + df_varieties['P(Drought Damage)']
ax2.scatter(risk_scores, exp_yields, s=200, c=['red', 'yellow', 'green'], 
           edgecolor='black', linewidth=2, alpha=0.7)
for i, var in enumerate(varieties_list):
    ax2.annotate(f'Variety {var}', (risk_scores[i], exp_yields[i]), 
                xytext=(10, 10), textcoords='offset points', fontweight='bold')
ax2.set_xlabel('Total Risk Score')
ax2.set_ylabel('Expected Yield')
ax2.set_title('Risk-Return Trade-off')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Decision 2: Irrigation Investment

Monte Carlo simulation to evaluate irrigation ROI

In [ ]:
# Parameters
acres = 200
irrigation_cost = 80000  # One-time investment
years_horizon = 10
n_simulations = 5000

print("Irrigation Investment Analysis (Monte Carlo)")
print("="*80)
print(f"Farm size: {acres} acres")
print(f"Irrigation cost: ${irrigation_cost:,}")
print(f"Analysis period: {years_horizon} years")
print(f"Simulations: {n_simulations:,}\n")

# Simulate without irrigation
yields_no_irr = stats.norm(95, 18).rvs((n_simulations, years_horizon))  # Higher variance
prices = stats.norm(5.5, 0.9).rvs((n_simulations, years_horizon))
costs_no_irr = stats.norm(280, 25).rvs((n_simulations, years_horizon))

revenue_no_irr = ((yields_no_irr * prices - costs_no_irr) * acres).sum(axis=1)

# Simulate with irrigation
yields_with_irr = stats.norm(108, 10).rvs((n_simulations, years_horizon))  # Higher mean, lower variance
costs_with_irr = costs_no_irr + 15  # Additional irrigation operating cost

revenue_with_irr = ((yields_with_irr * prices - costs_with_irr) * acres).sum(axis=1) - irrigation_cost

# Analysis
net_benefit = revenue_with_irr - revenue_no_irr

print(f"Results:")
print(f"  Mean benefit: ${net_benefit.mean():,.0f}")
print(f"  Std dev: ${net_benefit.std():,.0f}")
print(f"  P(positive return) = {(net_benefit > 0).mean():.1%}")
print(f"  P(return > $50k) = {(net_benefit > 50000).mean():.1%}")
print(f"\n  10th percentile: ${np.percentile(net_benefit, 10):,.0f}")
print(f"  90th percentile: ${np.percentile(net_benefit, 90):,.0f}")

if net_benefit.mean() > 0 and (net_benefit > 0).mean() > 0.65:
    print(f"\n✓ INVEST: Strong positive expected value")
else:
    print(f"\n✗ RISKY: Uncertain returns")

# Visualize
plt.figure(figsize=(12, 6))
plt.hist(net_benefit, bins=50, edgecolor='black', alpha=0.7, density=True)
plt.axvline(0, color='red', linestyle='--', linewidth=2, label='Break-even')
plt.axvline(net_benefit.mean(), color='blue', linestyle='--', linewidth=2, 
           label=f'Mean = ${net_benefit.mean():,.0f}')
plt.fill_betweenx([0, plt.ylim()[1]], 0, net_benefit.max(), alpha=0.2, color='green', label='Positive Return')
plt.xlabel('Net Benefit ($)')
plt.ylabel('Probability Density')
plt.title(f'Irrigation Investment: {years_horizon}-Year Net Benefit Distribution')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

print(f"\n💡 {(net_benefit > 0).sum():,} out of {n_simulations:,} simulations show positive returns")

## Decision 3: Crop Insurance

Expected value analysis for insurance purchase

In [ ]:
# Insurance options
revenue_per_acre = 100 * 5.5  # Expected: 100 bu/ac × $5.50/bu
total_expected_revenue = revenue_per_acre * acres

insurance_options = {
    'No Insurance': {'premium': 0, 'coverage': 0},
    'Basic (70%)': {'premium': 0.03 * total_expected_revenue, 'coverage': 0.70},
    'Standard (80%)': {'premium': 0.05 * total_expected_revenue, 'coverage': 0.80},
    'Premium (90%)': {'premium': 0.08 * total_expected_revenue, 'coverage': 0.90}
}

# Failure scenarios
P_total_failure = 0.05  # Catastrophic loss
P_partial_failure = 0.15  # 40-60% yield
P_success = 1 - P_total_failure - P_partial_failure

print("Crop Insurance Decision Analysis")
print("="*80)
print(f"Expected revenue: ${total_expected_revenue:,.0f}")
print(f"\nFailure Probabilities:")
print(f"  Total failure (0% revenue): {P_total_failure:.0%}")
print(f"  Partial failure (50% revenue): {P_partial_failure:.0%}")
print(f"  Success (100% revenue): {P_success:.0%}")

print(f"\nInsurance Options:")
results_insurance = []

for name, opt in insurance_options.items():
    premium = opt['premium']
    coverage = opt['coverage']
    
    # Expected net revenue
    # Success: full revenue - premium
    # Partial: 50% revenue + coverage payout - premium
    # Total: coverage payout - premium
    
    net_success = total_expected_revenue - premium
    net_partial = (0.50 * total_expected_revenue + 
                  max(0, coverage * total_expected_revenue - 0.50 * total_expected_revenue) - 
                  premium)
    net_total = coverage * total_expected_revenue - premium
    
    expected_net = (
        P_success * net_success +
        P_partial_failure * net_partial +
        P_total_failure * net_total
    )
    
    results_insurance.append({
        'Option': name,
        'Premium': premium,
        'Coverage': f'{coverage:.0%}',
        'Expected Net': expected_net
    })

df_insurance = pd.DataFrame(results_insurance)
print("\n" + df_insurance.to_string(index=False))

best_insurance = df_insurance.loc[df_insurance['Expected Net'].idxmax(), 'Option']
print(f"\n✓ RECOMMENDATION: {best_insurance}")

# Visualize
plt.figure(figsize=(10, 6))
options_list = [r['Option'] for r in results_insurance]
premiums = [r['Premium'] for r in results_insurance]
expected_nets = [r['Expected Net'] for r in results_insurance]

x = np.arange(len(options_list))
colors = ['red', 'orange', 'yellow', 'lightgreen']

bars = plt.bar(x, expected_nets, color=colors, edgecolor='black', linewidth=2, alpha=0.7)
plt.xticks(x, options_list, rotation=15)
plt.ylabel('Expected Net Revenue ($)')
plt.title('Insurance Options: Expected Value Comparison')
plt.grid(True, alpha=0.3, axis='y')

for i, (opt, net) in enumerate(zip(options_list, expected_nets)):
    plt.text(i, net + 1000, f'${net:,.0f}', ha='center', fontsize=9, fontweight='bold')

plt.tight_layout()
plt.show()

## Comprehensive Decision Summary

In [ ]:
print("="*80)
print("COMPREHENSIVE FARM PLAN RECOMMENDATIONS")
print("="*80)

print(f"\n1. CROP VARIETY: {best_variety}")
print(f"   Expected yield: {best_yield:.1f} bu/ac")
print(f"   Reasoning: Best risk-adjusted return")

print(f"\n2. IRRIGATION: {'INVEST' if net_benefit.mean() > 0 else 'DON\'T INVEST'}")
print(f"   Expected benefit: ${net_benefit.mean():,.0f} over {years_horizon} years")
print(f"   Probability of positive return: {(net_benefit > 0).mean():.0%}")

print(f"\n3. INSURANCE: {best_insurance}")
print(f"   Maximizes expected net revenue")

print(f"\n4. OVERALL STRATEGY:")
print(f"   • Plant {best_variety} variety for optimal risk-return")
print(f"   • {'Invest in' if net_benefit.mean() > 0 else 'Defer'} irrigation system")
print(f"   • Purchase {best_insurance} coverage")
print(f"   • Monitor weather and disease conditions closely")

print("\n" + "="*80)
print("\n💡 This plan integrates:")
print("   ✓ Conditional probability (disease and weather risks)")
print("   ✓ Expected value calculations (insurance, variety selection)")
print("   ✓ Monte Carlo simulation (irrigation investment)")
print("   ✓ Risk-return analysis (all decisions)")
print("="*80)

## Congratulations!

You've completed the **Probability Basics** module by applying ALL concepts to real farming decisions:

### Probability Concepts Used
- ✅ Sample spaces and events
- ✅ Probability axioms and rules
- ✅ Conditional probability
- ✅ Bayes' theorem
- ✅ Independence and dependence
- ✅ Law of total probability
- ✅ Expected value
- ✅ Monte Carlo simulation
- ✅ Risk-return trade-offs

### Skills Developed
- Data-driven decision making
- Quantifying uncertainty
- Economic optimization
- Risk management
- Probabilistic reasoning

### Real-World Applications
- Crop variety selection
- Investment decisions
- Insurance optimization
- Weather-based planning
- Disease management

## Next Steps in Your Learning Journey

You're ready for:
1. **Topic 02: Random Variables** - Formal treatment of probabilistic outcomes
2. **Topic 03: Common Distributions** - Normal, binomial, Poisson in depth
3. **Topic 04: Descriptive Statistics** - Summarizing agricultural data
4. **Machine Learning**: Apply these foundations to predictive models

### Remember
Probability is the language of uncertainty. You now speak that language fluently for agricultural applications!

**Well done!** 🎉